In [508]:
import pandas as pd

df = pd.read_csv("example/a1_raport.csv")

In [509]:
df.head()

,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,sredn._martinamm,sredn._sitowamm,srednia_jaskrawosc,r,g,b,y,liczenie_obiektow_w,udzial_punktow,pole_obrazu_mm2
0,1,292.0,699.0,29.199999999999999,69.900000000000006,0.92,1.3,0.8,69.239999999999995,4.08700571,...,1.3,1.07935823,128.000000000000000,97.500000000000000,139.800000000000011,147.199999999999989,128.000000000000000,1,12.199999999999999,9.000000000000000
1,2,572.0,615.5,57.200000000000003,61.600000000000001,1.31,1.4,1.3,115.359999999999999,4.26274164,...,1.4,1.29148899,139.699999999999989,105.900000000000006,152.500000000000000,162.400000000000006,139.699999999999989,1,13.400000000000000,11.199999999999999
2,3,971.0,137.5,97.099999999999994,13.800000000000001,0.65,1.1,0.7,61.009999999999998,3.15563488,...,1.1,0.90622266,120.599999999999994,88.900000000000006,132.699999999999989,141.099999999999994,120.599999999999994,1,9.800000000000001,8.100000000000000
3,4,867.5,241.5,86.799999999999997,24.199999999999999,0.28,0.7,0.6,81.500000000000000,2.07279218,...,0.7,0.59172703,143.099999999999994,112.400000000000006,155.300000000000011,160.900000000000006,143.099999999999994,1,5.500000000000000,6.800000000000000
4,5,833.5,535.5,83.400000000000006,53.600000000000001,0.97,1.4,1.1,21.719999999999999,4.06274165,...,1.4,1.11132460,130.599999999999994,98.400000000000006,143.300000000000011,149.400000000000006,130.599999999999994,1,12.000000000000000,9.600000000000000


In [510]:
# Scaling
df['dlugoscmm'] = df['dlugoscmm']/1.49
df['szerokoscmm'] = df['szerokoscmm']/1.49
df['powierzchniamm'] = df['powierzchniamm']/2.235

In [511]:
# Set proble volume in mL
PROBE_VOLUME_ML = 6

In [512]:
df.shape

(50, 23)

In [513]:
df = df[df['dlugoscmm'] >= 0.2]
df = df[df['szerokoscmm'] <= 1.5]

In [514]:
df.shape

(50, 23)

In [515]:
df['Pw'] = 3.14*(df['szerokoscmm']/2)**2+df['szerokoscmm']*(df['dlugoscmm']-df['szerokoscmm'])

In [516]:
df['R'] = df['Pw'] / df['powierzchniamm']

In [517]:
df['Dk'] = df['dlugoscmm']
df['Sk'] = df['szerokoscmm']
df['D/S'] = (df['Dk'] / df['Sk']).astype('float64')

In [518]:
bacteria_types = []
Dks = []
Sks = []
for idx,row  in df.iterrows():
    if row['R'] > 1.2:
        Dks.append(row['szerokoscmm'])
        Sks.append(row['Pw']*0.8)
        bacteria_types.append("Krzywe")
    else:
        Dks.append(row['dlugoscmm'])
        Sks.append(row['szerokoscmm'])
        if round(row['D/S'],15) > 1.5:
            bacteria_types.append("Pałeczki")
        else:
            bacteria_types.append("Ziarniaki")
df['bacteria_type'] = bacteria_types
df['Dk'] = Dks
df['Sk'] = Sks

In [519]:
df['Ob'] =(3.14*(df['Sk']**3)/6)+(3.14*((df['Sk']**2)/4)*(df['Dk']-df['Sk']))

In [520]:
df = df[df['Ob']>0]

In [521]:
df['biomasa'] =((104.5*(df['Ob']**0.59))*0.86)/1000000

In [522]:
df['bialko'] = 104.5 * (df['Ob']**0.59)

In [523]:
df['wegiel'] = 0.86

In [524]:
df['bacteria_type'].value_counts()

bacteria_type
Pałeczki     17
Ziarniaki    17
Krzywe       15
Name: count, dtype: int64

In [525]:
df.groupby("bacteria_type")

In [526]:
bins = [0, 0.1, 0.2, 0.5, 1.0, float("inf")]
labels = ["<=0.1", "0.1–0.2", "0.2–0.5", "0.5–1.0", ">1.0"]

df["Ob_bucket"] = pd.cut(df["Ob"], bins=bins, labels=labels, right=True)

grouped = df.groupby(["bacteria_type", "Ob_bucket"])
result = grouped.size().reset_index(name="count")

C:\Users\Kuba\AppData\Local\Temp\ipykernel_2776\3292400346.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(["bacteria_type", "Ob_bucket"])


In [527]:
df[df['bacteria_type']=='Pałeczki']

,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,R,Dk,Sk,D/S,bacteria_type,Ob,biomasa,bialko,wegiel,Ob_bucket
0,1,292.0,699.0,29.199999999999999,69.900000000000006,0.411633109619687,0.872483221476510,0.536912751677852,69.239999999999995,4.087005710000000,...,0.987452582433615,0.872483221476510,0.536912751677852,1.625000000000000,Pałeczki,0.156939138622351,0.000030136734205,35.042714192055946,0.86,0.1–0.2
2,3,971.0,137.5,97.099999999999994,13.800000000000001,0.290827740492170,0.738255033557047,0.469798657718121,61.009999999999998,3.155634880000000,...,1.029401135776975,0.738255033557047,0.469798657718121,1.571428571428572,Pałeczki,0.100776442845199,0.000023205801142,26.983489700078099,0.86,0.1–0.2
10,11,1245.0,652.0,124.500000000000000,65.200000000000003,0.138702460850112,0.536912751677852,0.335570469798658,82.629999999999995,2.214213530000000,...,1.124431695172115,0.536912751677852,0.335570469798658,1.600000000000000,Pałeczki,0.037573634498799,0.000012965695963,15.076390654891753,0.86,<=0.1
12,13,378.0,577.0,37.799999999999997,57.700000000000003,0.505592841163311,0.872483221476510,0.536912751677852,153.449999999999989,3.979898930000000,...,0.803943695432678,0.872483221476510,0.536912751677852,1.625000000000000,Pałeczki,0.156939138622351,0.000030136734205,35.042714192055946,0.86,0.1–0.2
14,15,828.5,776.0,82.900000000000006,77.599999999999994,0.152125279642058,0.604026845637584,0.335570469798658,83.500000000000000,2.355634870000000,...,1.173262929332807,0.604026845637584,0.335570469798658,1.800000000000000,Pałeczki,0.043506313630188,0.000014137108482,16.438498234884342,0.86,<=0.1
15,16,957.5,486.0,95.799999999999997,48.600000000000001,0.604026845637584,1.140939597315436,0.604026845637584,92.209999999999994,4.862741660000000,...,1.011073825503356,1.140939597315436,0.604026845637584,1.888888888888889,Pałeczki,0.269106325399817,0.000041425699097,48.169417554070186,0.86,0.2–0.5
16,17,887.5,438.0,88.799999999999997,43.799999999999997,0.290827740492170,0.872483221476510,0.402684563758389,59.960000000000001,3.321320300000000,...,1.088177594217863,0.872483221476510,0.402684563758389,2.166666666666667,Pałeczki,0.093973637441206,0.000022268363594,25.893446039444214,0.86,<=0.1
17,18,321.0,334.0,32.100000000000001,33.399999999999999,0.675615212527964,1.140939597315436,0.604026845637584,165.949999999999989,4.779898930000000,...,0.903940175118894,1.140939597315436,0.604026845637584,1.888888888888889,Pałeczki,0.269106325399817,0.000041425699097,48.169417554070186,0.86,0.2–0.5
19,20,1249.5,607.0,125.000000000000000,60.700000000000003,0.478747203579418,0.939597315436242,0.604026845637584,45.390000000000001,4.038477590000000,...,1.021623910180016,0.939597315436242,0.604026845637584,1.555555555555556,Pałeczki,0.211440684242714,0.000035931553098,41.780875695097528,0.86,0.2–0.5
21,22,1229.5,426.5,123.000000000000000,42.700000000000003,0.366890380313199,0.939597315436242,0.268456375838926,27.280000000000001,3.579898970000000,...,0.645277459486004,0.939597315436242,0.268456375838926,3.500000000000000,Pałeczki,0.048094252158462,0.000014998565207,17.440192101577484,0.86,<=0.1


In [528]:
df[(df['bacteria_type'] == 'Pałeczki') & (df['nr.'] == 35)]


,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,R,Dk,Sk,D/S,bacteria_type,Ob,biomasa,bialko,wegiel,Ob_bucket


In [529]:
df[df['bacteria_type']=='Krzywe']

,nr.,xpix.,ypix.,xmm,ymm,powierzchniamm,dlugoscmm,szerokoscmm,kat,obwodmm,...,R,Dk,Sk,D/S,bacteria_type,Ob,biomasa,bialko,wegiel,Ob_bucket
3,4,867.5,241.5,86.799999999999997,24.199999999999999,0.125279642058166,0.469798657718121,0.402684563758389,81.500000000000000,2.072792180000000,...,1.231783317353787,0.402684563758389,0.123453898473042,1.166666666666667,Krzywe,0.004325412557538,0.000003621354064,4.210876818654492,0.86,<=0.1
4,5,833.5,535.5,83.400000000000006,53.600000000000001,0.434004474272931,0.939597315436242,0.738255033557047,21.719999999999999,4.062741650000000,...,1.328288244655089,0.738255033557047,0.461186433043557,1.272727272727273,Krzywe,0.097594805198849,0.000022770705978,26.477565091106815,0.86,<=0.1
7,8,844.5,293.5,84.500000000000000,29.399999999999999,0.572706935123043,1.073825503355705,0.805369127516778,47.829999999999998,4.604162990000000,...,1.266568791946309,0.805369127516778,0.580298184766452,1.333333333333333,Krzywe,0.161762692694484,0.000030679832100,35.674223372558117,0.86,0.1–0.2
20,21,623.0,664.0,62.299999999999997,66.400000000000006,0.630872483221476,1.140939597315436,0.805369127516778,54.820000000000000,4.604162990000000,...,1.235470512637441,0.805369127516778,0.623539480203594,1.416666666666667,Krzywe,0.182369383896865,0.000032928836137,38.289344345286068,0.86,0.1–0.2
28,29,832.0,889.0,83.200000000000003,88.900000000000006,0.984340044742729,1.744966442953020,0.805369127516778,58.719999999999999,6.642640600000000,...,1.286028065893838,0.805369127516778,1.012711139137877,2.166666666666667,Krzywe,0.376616970516408,0.000050512129062,58.735033792845236,0.86,0.2–0.5
29,30,897.5,666.5,89.799999999999997,66.700000000000003,0.621923937360179,1.140939597315436,0.805369127516778,51.390000000000001,4.745584370000000,...,1.253247066776109,0.805369127516778,0.623539480203594,1.416666666666667,Krzywe,0.182369383896865,0.000032928836137,38.289344345286068,0.86,0.1–0.2
30,31,1026.5,837.5,102.700000000000003,83.799999999999997,0.129753914988814,0.536912751677852,0.335570469798658,82.640000000000001,2.048528120000000,...,1.201978708632261,0.335570469798658,0.124769154542588,1.600000000000000,Krzywe,0.003592551620708,0.000003245656253,3.774018898403065,0.86,<=0.1
35,36,856.0,753.5,85.599999999999994,75.400000000000006,0.116331096196868,0.469798657718121,0.402684563758389,130.740000000000009,1.907106770000000,...,1.326535880227155,0.402684563758389,0.123453898473042,1.166666666666667,Krzywe,0.004325412557538,0.000003621354064,4.210876818654492,0.86,<=0.1
37,38,324.5,616.5,32.500000000000000,61.700000000000003,0.344519015659955,0.805369127516778,0.671140939597315,140.400000000000006,3.472792180000000,...,1.287806153577965,0.671140939597315,0.354938966713211,1.200000000000000,Krzywe,0.054672257973695,0.000016176973647,18.810434472992004,0.86,<=0.1
38,39,464.0,36.5,46.399999999999999,3.700000000000000,0.129753914988814,0.469798657718121,0.469798657718121,161.919999999999987,2.072792180000000,...,1.335281184910900,0.469798657718121,0.138606369082474,1.000000000000000,Krzywe,0.006388345989275,0.000004558205648,5.300239125990047,0.86,<=0.1


In [530]:
result

,bacteria_type,Ob_bucket,count
0,Krzywe,<=0.1,9
1,Krzywe,0.1–0.2,3
2,Krzywe,0.2–0.5,2
3,Krzywe,0.5–1.0,1
4,Krzywe,>1.0,0
5,Pałeczki,<=0.1,6
6,Pałeczki,0.1–0.2,4
7,Pałeczki,0.2–0.5,4
8,Pałeczki,0.5–1.0,1
9,Pałeczki,>1.0,2


In [531]:
import numpy as np
total_bacteria_count = np.sum(result['count'])

In [532]:
scale_to_1_L = 1000 / PROBE_VOLUME_ML
result['count_in_1_L'] = result['count'] * scale_to_1_L

In [533]:

result['bio_diversity'] =((result['count']+1)/total_bacteria_count)*np.log((result['count']+1)/total_bacteria_count)

In [534]:

result['bio_diversity_1_L'] =((result['count_in_1_L']+1)/(total_bacteria_count*scale_to_1_L))*np.log((result['count_in_1_L']+1)/(total_bacteria_count*scale_to_1_L))

In [535]:
result

,bacteria_type,Ob_bucket,count,count_in_1_L,bio_diversity,bio_diversity_1_L
0,Krzywe,<=0.1,9,1500.000000000000000,-0.324333715329915,-0.311337286974257
1,Krzywe,0.1–0.2,3,500.000000000000000,-0.204532729550264,-0.171232189395515
2,Krzywe,0.2–0.5,2,333.333333333333314,-0.171012735271991,-0.130827128220269
3,Krzywe,0.5–1.0,1,166.666666666666657,-0.130558086430640,-0.079778637873300
4,Krzywe,>1.0,0,0.000000000000000,-0.079424904043074,-0.001102997890759
5,Pałeczki,<=0.1,6,1000.000000000000000,-0.277987164150759,-0.257284945699352
6,Pałeczki,0.1–0.2,4,666.666666666666629,-0.232896161803727,-0.204716987874147
7,Pałeczki,0.2–0.5,4,666.666666666666629,-0.232896161803727,-0.204716987874147
8,Pałeczki,0.5–1.0,1,166.666666666666657,-0.130558086430640,-0.079778637873300
9,Pałeczki,>1.0,2,333.333333333333314,-0.171012735271991,-0.130827128220269


In [536]:
shannon_index = np.sum(result['bio_diversity'])*-1
shannon_index_1_L = np.sum(result['bio_diversity_1_L'])*-1
print(f"Shannon index: {shannon_index}")
print(f"Shannon index 1 L: {shannon_index_1_L}")

Shannon index: 2.9511405104201236
Shannon index 1 L: 2.368357603731951
